<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/expression-language/LangChain_Expression_03_Router.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03 路由（Router）

`LEL` ( `LangChain Expression Language` ) 实现了路由机制，支持在应用开发中根据业务需要将请求转发给指定的链。

核心类：`RouterRunnable`

## 准备环境

安装必要的 `python` 包。

In [27]:
!pip install -q -U langchain openai

In [30]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap, Runnable, RunnableConfig, RunnablePassthrough, Input
from langchain.load.serializable import Serializable
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser
from operator import itemgetter
from typing import Optional, Dict

## 一个有用的调试类 `StdOutputRunnable`

类 `StdOutputRunnable` 的功能与 `RunnablePassthrough` 几乎一致，唯一的区别是打印出输入的数据。这非常适合于 `LEL` 的调试。将其加到管道中，可以查看所在环节的输入值，帮助调试。

In [24]:
class StdOutputRunnable(Serializable, Runnable[Input, Input]):
    @property
    def lc_serializable(self) -> bool:
        return True

    def invoke(self, input: Dict, config: Optional[RunnableConfig] = None) -> Input:
        print(input);
        return self._call_with_config(lambda x: x, input, config)


### RouterRunnable 示例

模型辅助类，在分类任务执行时，指定允许的分类值。

`create_tagging_chain_pydantic` 创建基于 `Pydantic` schema定义的分类链，请参考 [API文档](https://api.python.langchain.com/en/latest/chains/langchain.chains.openai_functions.tagging.create_tagging_chain_pydantic.html)

In [29]:
import os

os.environ['OPENAI_API_KEY'] = "您的有效openai api key"

In [40]:
from langchain.chains import create_tagging_chain_pydantic
from pydantic import BaseModel, Field

class ChainToUse(BaseModel):
    """Used to determine which chain to serve the user."""

    name: str = Field(description="Should be one of `color` or `fruit`")

tagger = create_tagging_chain_pydantic(ChainToUse, ChatOpenAI(temperature=0))

In [41]:
model = ChatOpenAI()

In [42]:
color_chain = ChatPromptTemplate.from_template("You are a color expert. Answer the question about color: {question}") | model
fruit_chain = ChatPromptTemplate.from_template("You are a fruit expert. Answer the question about fruit: {question}") | model

In [43]:
from langchain.schema.runnable import RouterRunnable
router = RouterRunnable({"color": color_chain, "fruit": fruit_chain})

In [46]:
chain = {
    "key": RunnablePassthrough() | tagger | StdOutputRunnable() | (lambda x: x['text'].name),
    "input": {"question": RunnablePassthrough()}
} | router

In [47]:
chain.invoke("What is the HEX code of color YELLOW?")

{'input': 'What is the HEX code of color YELLOW?', 'text': ChainToUse(name='color')}


AIMessage(content='The HEX code for the color yellow is #FFFF00.', additional_kwargs={}, example=False)

In [48]:
chain.invoke("What country grow most apples in 2000?")

{'input': 'What country grow most apples in 2000?', 'text': ChainToUse(name='fruit')}


AIMessage(content='In the year 2000, China was the country that grew the most apples.', additional_kwargs={}, example=False)